# Step 1: Import libraries


In [101]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np

## Step 2: Scrape data using BeautifulSoup and insert into pandas DF

In [102]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text

#making soup object
soup = bs(source, 'lxml')
# print(soup.prettify())

#table that we want to extract from
table = soup.find('table', class_='wikitable sortable')

#setting up lists to insert to dataframe later
A=[]
B=[]
C=[]

#iterate through rows and cells
for row in table.findAll('tr'):
    data = row.findAll('td')
    if len(data)==3:
        A.append(data[0].find(text=True).strip())
        B.append(data[1].find(text=True).strip())
        C.append(data[2].find(text=True).strip())

#import into pandas df
Canada_Postal_Codes = pd.DataFrame(np.column_stack([A,B,C]),
                                  columns=['Postcode','Borough','Neighborhood'])
# Canada_Postal_Codes.shape # rows before dropping borough "Not Assigned"

## Step 3: Removing values from DF, then grouping

In [103]:
#dropping not assigned borough  values
Canada_Postal_Codes = Canada_Postal_Codes[Canada_Postal_Codes.Borough != 'Not assigned']
# Canada_Postal_Codes.shape

In [104]:
#groupby postcode
Canada_Postal_Codes = Canada_Postal_Codes.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

## Step 4: Assign Borough values to Neighborhood if there's no assigned value in the column 

In [105]:
#assigning borough to neighborhood for not assigned values
New_Neighborhood = np.where(Canada_Postal_Codes.Neighborhood == 'Not assigned', Canada_Postal_Codes.Borough,Canada_Postal_Codes.Neighborhood)
Canada_Postal_Codes['New Neighborhood']=New_Neighborhood

In [106]:
# formatting
Canada_Postal_Codes.drop('Neighborhood', axis=1, inplace=True)

In [107]:
# formatting
Canada_Postal_Codes = Canada_Postal_Codes.rename(columns={'New Neighborhood': 'Neighborhood'})

## Step 5: Printing DF shape

In [108]:
Canada_Postal_Codes.shape

(103, 3)